# linux 1.0 notes

## net/inet/skbuff

```c
struct sk_buff {
  unsigned long			magic_debug_cookie;
  struct sk_buff		*volatile next;
  struct sk_buff		*volatile prev;
  struct sk_buff		*volatile link3;
  struct sk_buff		*volatile* list;
  struct sock			*sk;
  volatile unsigned long	when;	/* used to compute rtt's	*/
  struct device			*dev;
  void				*mem_addr;
  union {
	struct tcphdr	*th;
	struct ethhdr	*eth;
	struct iphdr	*iph;
	struct udphdr	*uh;
	struct arphdr	*arp;
	unsigned char	*raw;
	unsigned long	seq;
#ifdef CONFIG_IPX	
	ipx_packet	*ipx;
#endif	
  } h;
  struct iphdr		*ip_hdr;		/* For IPPROTO_RAW */
  unsigned long			mem_len;
  unsigned long 		len;
  unsigned long			fraglen;
  struct sk_buff		*fraglist;	/* Fragment list */
  unsigned long			truesize;
  unsigned long 		saddr;
  unsigned long 		daddr;
  int				magic;
  volatile char 		acked,
				used,
				free,
				arp;
  unsigned char			tries,lock;	/* Lock is now unused */
  unsigned short		users;		/* User count - see datagram.c (and soon seqpacket.c/stream.c) */
  unsigned long			padding[0];
  unsigned char			data[0];
};
```

* last field are data buffer. So this struct has a dynamic size. 

* 
![skbuff01.png](imgs/skbuff01.png)

------------

```c
/*
 *	Get a clone of an sk_buff. This is the safe way to peek at
 *	a socket queue without accidents. Its a bit long but most
 *	of it acutally ends up as tiny bits of inline assembler
 *	anyway. Only the memcpy of upto 4K with ints off is not
 *	as nice as I'd like.
 */

struct sk_buff *skb_peek_copy(struct sk_buff *volatile* list)
{
	struct sk_buff *orig,*newsk;
	unsigned long flags;
	unsigned int len;
	/* Now for some games to avoid races */

	do
	{
		save_flags(flags);
		cli();
		orig=skb_peek(list);
		if(orig==NULL)
		{
			restore_flags(flags);
			return NULL;
		}
		IS_SKB(orig);
		len=orig->truesize;
		restore_flags(flags);

		newsk=alloc_skb(len,GFP_KERNEL);	/* May sleep */

		if(newsk==NULL)		/* Oh dear... not to worry */
			return NULL;

		save_flags(flags);
		cli();
		if(skb_peek(list)!=orig)	/* List changed go around another time */
		{
			restore_flags(flags);
			newsk->sk=NULL;
			newsk->free=1;
			newsk->mem_addr=newsk;
			newsk->mem_len=len;
			kfree_skb(newsk, FREE_WRITE);
			continue;
		}

		IS_SKB(orig);
		IS_SKB(newsk);
		memcpy(newsk,orig,len);
		newsk->list=NULL;
		newsk->magic=0;
		newsk->next=NULL;
		newsk->prev=NULL;
		newsk->mem_addr=newsk;
		newsk->h.raw+=((char *)newsk-(char *)orig);
		newsk->link3=NULL;
		newsk->sk=NULL;
		newsk->free=1;
	}
	while(0);

	restore_flags(flags);
	return(newsk);
}

```

* There are a lot of `cli();` in this file. It is used to ensure the interupt flag is cleaned for some funcions will use `sti()` inside.

* `do{...}while(0)` nice trick. We can use `continue` inside and if the condition statisfied, it will go out from the loop. No need a `break`

* `newsk=alloc_skb(len,GFP_KERNEL);	/* May sleep */` when use alloc memory, we may swap out some memory page, this will cause a sleep because of writing to disk.

* volatile and pointer

![skbuff02.png](imgs/skbuff02.png)
![skbuff03.png](imgs/skbuff03.png)

So `struct sk_buff *volatile* list` means

`*list` is volatile, which is the the sk_buff pointer, but the pointer ot pointer `**list` is not.

------------